In [27]:
require 'torch'

local TTensor = {}

function TTensor.hosvd()

end

-- Khatri-rao product required for Tensor decomposition
function TTensor.krp()

end

-- Kronecker product
function TTensor.kroneckerp()

end

a = torch.Tensor(3,3,3)
b = torch.Tensor(3,3,3)
i = 0
a:apply(function()
        i = i+1
       return i
       end)

b:apply(function()
        i = i+1
       return i
       end)
print(a)
print(b)
c = a + b


(1,.,.) = 
   1   2   3
   4   5   6
   7   8   9

(2,.,.) = 
  10  11  12
  13  14  15
  16  17  18

(3,.,.) = 
  19  20  21
  22  23  24
  25  26  27
[torch.DoubleTensor of size 3x3x3]

(1,.,.) = 
  28  29  30
  31  32  33
  34  35  36

(2,.,.) = 
  37  38  39
  40  41  42
  43  44  45

(3,.,.) = 
  46  47  48
  49  50  51
  52  53  54
[torch.DoubleTensor of size 3x3x3]



In [7]:
require 'torch'   -- torch
require 'image'   -- for color transforms



-- parse command line arguments
if not opt then
   print '==> processing options'
   cmd = torch.CmdLine()
   cmd:text()
   cmd:text('TensorFaces')
   cmd:text()
   cmd:text('Options:')
   cmd:option('-size', 'small', 'how many people do we load: small(5)| medium(10) | full')
   cmd:option('-visualize', true, 'visualize input data and intermediate steps')
   cmd:text()
   opt = cmd:parse(arg or {})
end

----------------------------------------------------------------------
print '==> downloading dataset'
www = 'http://www.wisdom.weizmann.ac.il/~vision/FaceBase/'
train_file = 'FaceBase_png.zip'

if not paths.filep(train_file) then
   os.execute('wget ' .. www .. train_file)
   os.execute('unzip ' .. train_file)
end

--- Default size of the Weizmann data set its 28 people
n_people = 28
if (opt.size == 'small') then
    print '==> small dataset'
    n_people = 5
elseif (opt.size == 'medium') then
    print '==> medium dataset'
    n_people = 10
elseif (opt.size == 'large') then
    print '==> large/default dataset'
end

print '==> loading dataset'
train_dir = paths.cwd() .. '/FaceBase_png'
--- read_data
print '==> reading data into tensor'

-- List all the files in the directory, they are in the format:
-- amit-vp0-il0-ex2.png : name-pose<id>-illumination<id>-expression<id>.png
-- reading all the files in and creating a 5D tensor:
-- people x 6 poses x 3 illum x 2 expression x pixels
-- name : name of the person
-- pose : left,right etc
-- illu : illumination
-- expr : expression of the person
-- pix_r/_c : pixels row/colum
--        name     pose     illu       expr      pix_r     pix_c   
f_dict = {[1] = {},[2] = {}, [3] = {}, [4] = {}, [5] = {}, [6] = {}}
dtensor = {}

function string:split(sep)
    local sp, fields = sep or '-', {}
    local pattern = string.format("([^%s]+)", sp)
    self:gsub(pattern, function(c) fields[#fields + 1] = c end)
    return fields
end

-- Append the file characteristics to the table maintaining all the meta information
-- Here assuming that t and f fields are in order
function append_fileinfo(f_dict, f)
    local i = 1
    for fi, fv in ipairs(f) do
        local tt = f_dict[fi]
        if tt == nil then
            table.insert(tt, fv, 0)
        elseif tt[fv] == nil then
            tt[fv] = 0
        else
            tt[fv] = tt[fv] + 1
        end
    end
end

function table_size(t)
    count = 0
    for ti, tv in pairs(t) do
        count = count + 1
    end
    return count
end

for f in paths.iterfiles(train_dir) do
    if (string.find(f,'png') ~= nil) then
        f_list = f:split()
        append_fileinfo(f_dict, f_list)
    end
end
    
num_name = table_size(f_dict[1])
num_pose = table_size(f_dict[2])
num_illu = table_size(f_dict[3])
num_expr = table_size(f_dict[4])
num_pixels = 512*352
-- D = torch.Tensor(num_name, num_pose, num_illu, num_expr, num_pixels)
Dorig = torch.Tensor(num_name*num_pose*num_illu*num_expr, num_pixels)

function rgb2gray(im)
    local colorbyte = torch.Tensor(512, 352)

    for i = 1, im:size()[2] do
        for j = 1, im:size()[3] do
            colorbyte[i][j] = 0.21* im[1][i][j] + 0.72*im[2][i][j] + 0.07*im[3][i][j]
        end
    end
    return colorbyte
end

Dcounter = 1
print(train_dir)
for f in paths.iterfiles(train_dir) do
    if (string.find(f, 'png') ~= nil) then
        print (f)
        local im = image.load(train_dir .. '/' .. f, 'byte')
        Dorig[Dcounter] = torch.reshape(rgb2gray(im), num_pixels)
        Dcounter = Dcounter + 1
        print (Dorig:size())
    end
end
    
D = torch.reshape(Dorig, num_name, num_pose, num_illu, num_expr, num_pixels)


==> downloading dataset	
==> small dataset	
==> loading dataset	
==> reading data into tensor	


/home/kulinseth/ml/tensors/FaceBase_png	
uri-vp1-il1-ex3.png	



    840
 180224
[torch.LongStorage of size 2]



shmuel-vp1-il0-ex2.png	



    840
 180224
[torch.LongStorage of size 2]

meir-vp0-il0-ex2.png	


...

yishay-vp3-il2-ex2.png	



    840
 180224
[torch.LongStorage of size 2]

eli-vp1-il0-ex3.png	



    840
 180224
[torch.LongStorage of size 2]



moshe-vp1-il0-ex2.png	



    840
 180224
[torch.LongStorage of size 2]



In [9]:
print (D:size())

DEF_MAXITER = 500
DEF_CONV = 1e-7

--- N : number of dimensions
--- rank : rank of the Tensor
--- dtype : type of the tensor
function init(X, rank):
    -- Don't compute initial factor for first index, gets computed in
    -- first iteration
    return hosvd(X, rank, False)
end

function Tunfold
function nvecs(X, n, rank, do_flipsign=True)
    ---
    --- Eigendecomposition of mode-n unfolding of a tensor
    ---
    Xn = X.unfold(n)
    --if issparse_mat(Xn):
    --    Xn = csr_matrix(Xn, dtype=dtype)
    --    Y = Xn.dot(Xn.T)
    --    _, U = eigsh(Y, rank, which='LM')
    --else:
    Y = Xn.dot(Xn.T)
    N = Y:size(0)
    _, U = torch.eig(Y, eigvals=(N - rank, N - 1))
        #_, U = eigsh(Y, rank, which='LM')
    # reverse order of eigenvectors such that eigenvalues are decreasing
    U = array(U[:, ::-1])
    -- flip sign
    --if do_flipsign:
    --  U = flipsign(U)
    return U


        
function hooi(X, rank)
---
--- Compute Tucker decomposition of a tensor using Higher-Order Orthogonal
--- Iterations.
--- Parameters
    ----------
--- X : The tensor to be decomposed
--- rank : array_like
---     The rank of the decomposition for each mode of the tensor.
---     The length of ``rank`` must match the number of modes of ``X``.
--- init : {'random', 'nvecs'}, optional
---     The initialization method to use.
---         - random : Factor matrices are initialized randomly.
---         - nvecs : Factor matrices are initialzed via HOSVD.
---     default : 'nvecs'
--- Examples
    --------
--- Create dense tensor
--- >>> T = np.zeros((3, 4, 2))
--- >>> T[:, :, 0] = [[ 1,  4,  7, 10], [ 2,  5,  8, 11], [3,  6,  9, 12]]
--- >>> T[:, :, 1] = [[13, 16, 19, 22], [14, 17, 20, 23], [15, 18, 21, 24]]
--- >>> T = dtensor(T)
--- Compute Tucker decomposition of ``T`` with n-rank [2, 3, 1] via higher-order
--- orthogonal iterations
--- >>> Y = hooi(T, [2, 3, 1], init='nvecs')
--- Shape of the core tensor matches n-rank of the decomposition.
--- >>> Y['core'].shape
--- (2, 3, 1)
--- >>> Y['U'][1].shape
--- (3, 2)
--- References
    ----------
--- .. [1] L. De Lathauwer, B. De Moor, J. Vandewalle: On the best rank-1 and
---        rank-(R_1, R_2, \ldots, R_N) approximation of higher order tensors;
---        IEEE Trans. Signal Process. 49 (2001), pp. 2262-2271
---  init options
    local maxIter = DEF_MAXITER
    local conv    = DEF_CONV
    local dtype   = X:type()

    local ndims = X:nDimensions()
    if (rank == math.floor(rank)):
        rank = rank * ones(ndims)

    normX = torch.norm(X)

    U = init(X, rank)
    fit = 0
    exectimes = []
    for itr in (1, maxIter) do
        fitold = fit

        for n in (1, ndims):
            Utilde = ttm(X, U, n, transp=True, without=True)
            U[n] = nvecs(Utilde, n, rank[n])

        -- compute core tensor to get fit
        core = ttm(Utilde, U, n, transp=True)

        -- since factors are orthonormal, compute fit on core tensor
        normresidual = sqrt(normX ** 2 - norm(core) ** 2)

        -- fraction explained by model
        fit = 1 - (normresidual / normX)
        fitchange = abs(fitold - fit)
        if itr > 1 and fitchange < conv:
            break
    end
    return core, U
end
    
function hosvd(X, rank, dims=None, dtype=None, compute_core=True)
    U = [None for _ in range(X.ndim)]
    if dims is None:
        dims = range(X.ndim)
    if dtype is None:
        dtype = X.dtype
    for d in dims:
        U[d] = array(nvecs(X, d, rank[d]), dtype=dtype)
    if compute_core:
        core = X.ttm(U, transp=True)
        return U, core
    else:
        return U
                            
                            
local TTensor = {}

function TTensor.hosvd(X, rank, dims=None, dtype=None, compute_core=True)
    U = [None for _ in range(X.ndim)]
    if dims is None:
        dims = range(X.ndim)
    if dtype is None:
        dtype = X.dtype
    for d in dims:
        U[d] = array(nvecs(X, d, rank[d]), dtype=dtype)
    if compute_core:
        core = X.ttm(U, transp=True)
        return U, core
    else:
        return U

end

-- Khatri-rao product required for Tensor decomposition
function TTensor.krp()

end

-- Kronecker product
function TTensor.kroneckerp()

end

a = torch.Tensor(3,3,3)
b = torch.Tensor(3,3,3)
i = 0
a:apply(function()
        i = i+1
       return i
       end)

b:apply(function()
        i = i+1
       return i
       end)
print(a)
print(b)
c = a + b



     28
      5
      3
      2
 180224
[torch.LongStorage of size 5]

